In [1]:
# import stuff

%matplotlib inline 

# libraries
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import numpy.random as rng
np.set_printoptions(precision=6)
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor

#metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from time import time

# Fixing random state for reproducibility
np.random.seed(10)

In [2]:
print("CROW MODE \U0001F54A")

CROW MODE 🕊


In [3]:
df = pd.read_csv('a4-data\part1regression\diamonds.csv')

In [4]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326
1,2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326
2,3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327
3,4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334
4,5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335


In [5]:
# check for missing vals
df.isna().any()

Unnamed: 0    False
carat         False
cut           False
color         False
clarity       False
depth         False
table         False
x             False
y             False
z             False
price         False
dtype: bool

In [6]:
# drop the unnecessary index column
df.drop('Unnamed: 0',axis='columns',inplace=True)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335


In [7]:
# perform onehot encoding
cols = ['cut','color','clarity']
df_encoded = pd.get_dummies(df, columns=cols)

In [8]:
df_encoded.shape

(53940, 27)

In [9]:
df_encoded.head

<bound method NDFrame.head of        carat  depth  table     x     y     z  price  cut_Fair  cut_Good  \
0       0.23   61.5   55.0  3.95  3.98  2.43    326     False     False   
1       0.21   59.8   61.0  3.89  3.84  2.31    326     False     False   
2       0.23   56.9   65.0  4.05  4.07  2.31    327     False      True   
3       0.29   62.4   58.0  4.20  4.23  2.63    334     False     False   
4       0.31   63.3   58.0  4.34  4.35  2.75    335     False      True   
...      ...    ...    ...   ...   ...   ...    ...       ...       ...   
53935   0.72   60.8   57.0  5.75  5.76  3.50   2757     False     False   
53936   0.72   63.1   55.0  5.69  5.75  3.61   2757     False      True   
53937   0.70   62.8   60.0  5.66  5.68  3.56   2757     False     False   
53938   0.86   61.0   58.0  6.15  6.12  3.74   2757     False     False   
53939   0.75   62.2   55.0  5.83  5.87  3.64   2757     False     False   

       cut_Ideal  ...  color_I  color_J  clarity_I1  clarity_IF  clar

### Split training/test data

In [10]:
#split
trX, teX, trY, teY = train_test_split(df_encoded.drop('price',axis='columns'),df_encoded['price'], test_size = 0.3, random_state = 309)

### Scale the Sets

In [11]:
# define the columns to be scaled
cols_to_scale = ['carat', 'depth', 'table', 'x', 'y', 'z']

# create a column transformer object
ct = ColumnTransformer(
    transformers=[
        ('StandardScaler', StandardScaler(), cols_to_scale)
    ], remainder='passthrough'
)


In [12]:
# fit and transform the data
trX_scaled = ct.fit_transform(trX)
teX_scaled = ct.transform(teX)

In [13]:
trX_scaled = pd.DataFrame(trX_scaled,columns=trX.columns)
teX_scaled = pd.DataFrame(teX_scaled,columns=teX.columns)

In [14]:
print(trX_scaled.shape)
print(teX_scaled.shape)

(37758, 26)
(16182, 26)


### Based on exploratory data analysis, discuss what preprocessing that you need to do before regression, and provide evidence and justifications.
- Preprocessing includes the onehot encoding of categorical columns and the scaling of numeric columns

# Regression Algorithms

### Dictionary of Algs


In [15]:
algs = {
    'Linear Regression':LinearRegression(),
    'KNeighbours':KNeighborsRegressor(),
    'Ridge Regression':Ridge(),
    'Decision Tree':DecisionTreeRegressor(),
    'Random Forest':RandomForestRegressor(),
    'Gradient Boosting':GradientBoostingRegressor(),
    'SGD Regressor':SGDRegressor(),
    'SVR':SVR(),
    'Linear SVR':LinearSVR(),
    'MLP':MLPRegressor()
}

### Define a Relative Squared Error function

In [16]:
def relative_squared_error(true_y,predicted_y):
    true_mean = np.mean(true_y)
    mse = np.sum(np.square(true_y-predicted_y))
    mse_truemean = np.sum(np.square(true_y-true_mean))
    return mse/mse_truemean

### create dataframe for model performance

In [17]:
perf_df = pd.DataFrame(columns=['Algorithm','MSE','RMSE','RSE','MAE','Performance Time'])

### run metrics on different regression algorithms

In [18]:
for key, model in algs.items():
    # Fit the model on the training data
    model.fit(trX_scaled, trY)

    start_time = time()  # Record start time
    
    # Predict on the test data
    predictions = model.predict(teX_scaled)
    
    end_time = time()  # Record end time

    # Calculate regression metrics
    mse = mean_squared_error(teY, predictions)
    rmse = mean_squared_error(teY, predictions, squared=False)
    rse = relative_squared_error(teY, predictions)
    mae = mean_absolute_error(teY, predictions)

    # Calculate execution time
    perfTime = end_time - start_time

    # store results for each algorithm      
    row = {'Algorithm':{key},'MSE':mse,'RMSE':rmse,'RSE':rse,'MAE':mae,'Performance Time':perfTime}
    perf_df.loc[len(perf_df)] = row

C:\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Table of Performance - Regression Algorithms

In [19]:
perf_df.round(2).head(10)

,Algorithm,MSE,RMSE,RSE,MAE,Performance Time
0,{Linear Regression},1324176.56,1150.73,0.08,743.19,0.00
1,{KNeighbours},660521.13,812.72,0.04,415.94,0.92
2,{Ridge Regression},1324229.91,1150.75,0.08,743.18,0.00
3,{Decision Tree},532345.72,729.62,0.03,353.47,0.02
4,{Random Forest},302534.65,550.03,0.02,273.11,0.53
5,{Gradient Boosting},554737.42,744.81,0.03,406.42,0.03
6,{SGD Regressor},1329333.24,1152.97,0.08,742.00,0.00
7,{SVR},8313686.10,2883.35,0.51,1373.55,80.44
8,{Linear SVR},2455483.77,1567.00,0.15,819.60,0.00
9,{MLP},533487.99,730.40,0.03,373.70,0.01


In [20]:
perf_df.drop(columns='Algorithm').idxmax(axis=0)

MSE                 7
RMSE                7
RSE                 7
MAE                 7
Performance Time    7
dtype: int64

In [21]:
perf_df.drop(columns='Algorithm').idxmin(axis=0)

MSE                 4
RMSE                4
RSE                 4
MAE                 4
Performance Time    0
dtype: int64

From the above analysis, we can see that index 7 (Support Vector Regression) had the highest errors and took the longest. Random Forest Regressor had the lowest errors but was third-slowest, and Ridge Regression was the fastest but had 4th-highest errors. 